In [1]:
import pandas as pd

# --- CONFIGURAÇÃO ---
arquivo_original_csv = 'ai_job_market.csv'
arquivo_dicionario_csv = 'dicionario_traducao.csv'

# Carregar o dataset original
print(f"Lendo o arquivo original: '{arquivo_original_csv}'...")
df = pd.read_csv(arquivo_original_csv)

# Lista para armazenar todos os termos que precisam de tradução
termos_para_traduzir = []

# 1. Adicionar os nomes das colunas à lista
for coluna in df.columns:
    termos_para_traduzir.append({
        'Tipo': 'Coluna',
        'Termo_Original': coluna,
        'Traducao': '' # Deixamos em branco para você preencher
    })

# 2. Adicionar os valores únicos das colunas de texto (object)
for coluna in df.select_dtypes(include=['object']).columns:
    for valor_unico in df[coluna].dropna().unique():
        termos_para_traduzir.append({
            'Tipo': 'Dado',
            'Termo_Original': valor_unico,
            'Traducao': '' # Deixamos em branco para você preencher
        })

# Criar um DataFrame com os termos
df_dicionario = pd.DataFrame(termos_para_traduzir)

# Remover duplicatas caso um valor de dado seja igual a um nome de coluna
df_dicionario = df_dicionario.drop_duplicates(subset=['Termo_Original'])

# Salvar o DataFrame em um novo arquivo CSV que será o seu dicionário
df_dicionario.to_csv(arquivo_dicionario_csv, index=False)

print(f"\n✅ Arquivo '{arquivo_dicionario_csv}' criado com sucesso!")
print("Agora, abra este arquivo e preencha a coluna 'Traducao'.")

# Mostra as primeiras linhas do arquivo criado para você ver como ficou
print("\nAmostra do dicionário criado:")
print(df_dicionario.head())

Lendo o arquivo original: 'ai_job_market.csv'...

✅ Arquivo 'dicionario_traducao.csv' criado com sucesso!
Agora, abra este arquivo e preencha a coluna 'Traducao'.

Amostra do dicionário criado:
     Tipo   Termo_Original Traducao
0  Coluna           job_id         
1  Coluna     company_name         
2  Coluna         industry         
3  Coluna        job_title         
4  Coluna  skills_required         


In [3]:
import pandas as pd
from sqlalchemy import create_engine

# --- CONFIGURAÇÃO ---
# Certifique-se que os nomes dos arquivos estão corretos
arquivo_original_csv = 'ai_job_market.csv' 
arquivo_dicionario_csv = 'dicionario_traducao.csv'
arquivo_banco_dados_db = 'catalogo_vagas.db' # Usando o nome que sugerimos
nome_tabela_db = 'vagas_de_emprego'

# 1. Carregar o DataFrame original e o dicionário preenchido
print("Lendo o arquivo original e o dicionário de tradução...")
df_original = pd.read_csv(arquivo_original_csv)
df_dicionario = pd.read_csv(arquivo_dicionario_csv)

# 2. <<< PASSO EXTRA: PREENCHER TRADUÇÕES VAZIAS! >>>
# ===================================================================
print("Verificando e preenchendo traduções vazias com o termo original...")
# Esta linha encontra qualquer 'NaN' na coluna 'Traducao' e usa o valor
# da coluna 'Termo_Original' para preencher, garantindo que nada fique nulo.
df_dicionario['Traducao'].fillna(df_dicionario['Termo_Original'], inplace=True)
# ===================================================================

# 3. Criar os mapas de tradução a partir do dicionário (agora seguro)
print("Criando mapas de tradução...")
mapa_colunas = df_dicionario[df_dicionario['Tipo'] == 'Coluna'].set_index('Termo_Original')['Traducao'].to_dict()
mapa_dados = df_dicionario[df_dicionario['Tipo'] == 'Dado'].set_index('Termo_Original')['Traducao'].to_dict()

# 4. Aplicar as traduções
print("Aplicando traduções em massa no DataFrame...")
df_traduzido = df_original.rename(columns=mapa_colunas)
df_traduzido = df_traduzido.replace(mapa_dados)

# 5. A CORREÇÃO ESPECIAL para "Média" vs "Pleno"
print("Aplicando correção específica na coluna 'tamanho_empresa'...")
df_traduzido['tamanho_empresa'] = df_traduzido['tamanho_empresa'].replace({'Pleno': 'Média'})

print("\nDados traduzidos e corrigidos com sucesso! Amostra:")
print(df_traduzido.head())

# 6. Salvar o DataFrame final no banco de dados SQLite
print(f"\nSalvando dados no banco de dados '{arquivo_banco_dados_db}'...")
engine = create_engine(f'sqlite:///{arquivo_banco_dados_db}')
df_traduzido.to_sql(nome_tabela_db, engine, if_exists='replace', index=False)

print(f"\n🚀 Processo finalizado! Seus dados foram salvos na tabela '{nome_tabela_db}'.")

Lendo o arquivo original e o dicionário de tradução...
Verificando e preenchendo traduções vazias com o termo original...
Criando mapas de tradução...
Aplicando traduções em massa no DataFrame...


C:\Users\Guilh\AppData\Local\Temp\ipykernel_14020\3363928589.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dicionario['Traducao'].fillna(df_dicionario['Termo_Original'], inplace=True)


Aplicando correção específica na coluna 'tamanho_empresa'...

Dados traduzidos e corrigidos com sucesso! Amostra:
   id_vaga              nome_empresa       setor  \
0        1           Foster and Sons       Saúde   
1        2   Boyd, Myers and Ramirez  Tecnologia   
2        3                  King Inc  Tecnologia   
3        4  Cooper, Archer and Lynch  Tecnologia   
4        5                  Hall LLC    Finanças   

                               cargo  \
0                  Analista de Dados   
1  Engenheiro de Visão Computacional   
2           Pesquisador Quantitativo   
3           Gerente de Produto de IA   
4                 Cientista de Dados   

                             habilidades_necessarias nivel_experiencia  \
0  NumPy, Reinforcement Learning, PyTorch, Scikit...             Pleno   
1                    Scikit-learn, CUDA, SQL, Pandas            Sênior   
2          MLflow, FastAPI, Azure, PyTorch, SQL, GCP            Júnior   
3       Scikit-learn, C++, Pandas, L